In [56]:
#Libraries
from pyspark import SparkContext # spark
from pyspark.streaming import StreamingContext # spark streaming
import sys
import pymongo
from pymongo import MongoClient
from pprint import pprint #for printing Mongodb output in the queries
import json
import datetime as dt
import pandas as pd

In [57]:
client = MongoClient () #defining the Mongodb client.
result = client.drop_database('as2TaskB') #ensure that the as2TaskB database 
                                          #doesn't already exist

db = client.as2TaskB #defining the db

fireCollection = db.fire #define a new collection for fire data. 
                         #This will store fire data plus the 
                         #the embedded climate data associated 
                         #with each fire

climateCollection = db.climate #define a new collection 
                               #for the climate data

result = fireCollection.drop() #Ensure that the fire collection 
                               #does not already exist in Mongo db

result = climateCollection.drop() #Ensure that the fire collection 
                                  #does not already exist in Mongo db

In [59]:
def sendPartition(iter):
    connection = MongoClient()
    db = connection.get_database('as2TaskB')
    climateCollection = db.climate
    fireCollection = db.fire
    
    #collection.insert_many(climateRecord)
    for record in iter:
        #print(record)
        recType = record[0]
 
        if recType == "'cdata'": 
            station = int(record[1])
            air_temp = int(record[2])
            rel_humidity = float(record[3])
            windspeed_knots = float(record[4])
            maxwind_speed = float(record[5])
            precipitation = record[6]
            climateDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")), 'Station':station, 'Air_Temperature_Celcius':air_temp, 'Relative_Humidity':rel_humidity, 'WindSpeed_knots':windspeed_knots, 'Max_Wind_Speed':maxwind_speed, 'Precipitation ':precipitation}, index=[0])
            climateRecord = json.loads(climateDf.to_json(orient='records')) 
            print(climateRecord)
            climateCollection.insert_many(climateRecord)
        if recType == "'fdata'": 
            latitude = float(record[1])
            longitude = float(record[2])
            surfacetemp_kelvin = float(record[3])
            power = float(record[4])
            confidence = int(record[5])
            surfacetemp_celcius = int(record[6])
            fireDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Datetime':str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")), 'Latitiude':latitude, 'Longitude':longitude, 'Surface_Temperature_Kelvin':surfacetemp_kelvin, 'Power':power, 'Confidence':confidence, 'Surface_Temperature_Celcius ':surfacetemp_celcius}, index=[0])
            #fireDf["Climate"] = climateRecord
            fireRecord = json.loads(fireDf.to_json(orient='records'))
            print(fireRecord)
            fireCollection.insert_many(fireRecord)        
    connection.close()   

#("'2018-10-07 16:31:42'", ["'cdata'", '948701', '9', '43.2', '4.9', '8.0', "' 0.00I'", "'2018-10-07 16:31:42'"])

def sendNewPartition(iter):
    #str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    connection = MongoClient()
    db = connection.get_database('as2TaskB')
    climateCollection = db.climate
    fireCollection = db.fire
    
    for record in iter:
        #print(record)
        datestamp = record[0]
        climateRec = record[1]
        #fireRec = record[1][1]
    
        recType = climateRec[0]
 
        if recType == "'cdata'": 
            station = int(climateRec[1])
            air_temp = int(climateRec[2])
            rel_humidity = float(climateRec[3])
            windspeed_knots = float(climateRec[4])
            maxwind_speed = float(climateRec[5])
            precipitation = climateRec[6]
            climateDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Station':station, 'Air_Temperature_Celcius':air_temp, 'Relative_Humidity':rel_humidity, 'WindSpeed_knots':windspeed_knots, 'Max_Wind_Speed':maxwind_speed, 'Precipitation ':precipitation}, index=[0])
            climateRecord = json.loads(climateDf.to_json(orient='records')) 
            print(climateRecord)
            climateCollection.insert_many(climateRecord)
        if recType == "'fdata'": 
            latitude = float(fireRec[1])
            longitude = float(fireRec[2])
            surfacetemp_kelvin = float(fireRec[3])
            power = float(fireRec[4])
            confidence = int(fireRec[5])
            surfacetemp_celcius = int(fireRec[6])
            fireDf = pd.DataFrame({'Date':str(dt.datetime.now().strftime("%Y-%m-%d")), 'Datetime':str(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")), 'Latitiude':latitude, 'Longitude':longitude, 'Surface_Temperature_Kelvin':surfacetemp_kelvin, 'Power':power, 'Confidence':confidence, 'Surface_Temperature_Celcius ':surfacetemp_celcius}, index=[0])
            fireDf["Climate"] = climateRecord
            fireRecord = json.loads(fireDf.to_json(orient='records'))
            print(fireRecord)
            fireCollection.insert_many(fireRecord)        
    connection.close()   
       

In [28]:
#S1 = ssc.socketTextStream("localhost", 9999)
#S2 = ssc.socketTextStream("localhost", 8085)

    # Create windowed stream
#    wS1 = S1.window(10)
#    wS2 = S2.window(1)

#    wS1.flatMap(lambda line: line.split(",")).pprint()
#    wS2.flatMap(lambda line: line.split(",")).pprint()

    # Perform join
#    joinedStream = wS1.join(wS2)

#    joinedStream.foreachRDD(lambda rdd: rdd.foreach(lambda x: print(x)))


#rdd = sc.parallelize([("red",20),("red",30),("blue", 100)])
#rdd2 = sc.parallelize([("red",40),("red",50),("yellow", 10000)])
#rdd.join(rdd2).collect()
# Gives [('red', (20, 40)), ('red', (20, 50)), ('red', (30, 40)), ('red', (30, 50))]

#("'2018-10-07 16:31:42'", ["'cdata'", '948701', '9', '43.2', '4.9', '8.0', "' 0.00I'", "'2018-10-07 16:31:42'"])

In [60]:
# We add this line to avoid an error : "Cannot run multiple SparkContexts at once". If there is an existing spark context, we will reuse it instead of creating a new context.
sc = SparkContext.getOrCreate()

# Create a local StreamingContext with as many working processors as possible and a batch interval of 10 seconds            
batch_interval = 5

# If there is no existing spark context, we now create a new context
if (sc is None):
    sc = SparkContext(master="local[3]", appName = "WordCountApp")
ssc = StreamingContext(sc, batch_interval)

host = "localhost"
port1 = "8085"
port2 = "9997"

#lines = ssc.socketTextStream(host, int(port))
climateStream = ssc.socketTextStream(host, int(port1))
#fireStream = ssc.socketTextStream(host, int(port2))

#Create windows for streams
wclimateStream = climateStream.window(5)
#wfireStream = fireStream.window(5)

climateRecs = wclimateStream.map(lambda rec: rec[1:-1].split(", ")).map(lambda rec: (rec[7], rec))
#fireRecs = fireStream.map(lambda rec: rec[1:-1].split(", ")).map(lambda rec: (rec[7], rec))
#joinedStream = climateRecs.join(fireRecs)

#joinedStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
#joinedStream.pprint()

# Split each line into words
#words = lines.flatMap(lambda line: line.split(" "))
#climateRecs = climateStream.map(lambda rec: rec[1:-1].split(", "))
#fireRecs = fireStream.map(lambda rec: rec[1:-1].split(", "))

#climateRecs = climateStream.map(lambda rec: rec[1:-1].split(", "))
#fireRecs = fireStream.map(lambda rec: rec[1:-1].split(", "))
#datestamp = dt.datetime.now()
#cRdds = climateRecs.map(lambda rec: (datestamp, rec))
#fRdds = fireRecs.map(lambda rec: (datestamp, rec))
#jointRecs = joinedStream.map(lambda rec: rec[1:-1].split(", "))

#jointRecs = climateRecs.join(fireRecs)

climateRecs.foreachRDD(lambda rdd: rdd.foreachPartition(sendNewPartition))
#fireRecs.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))

# Count each word in each batch
#pairs = words.map(lambda word: (word, 1))
#wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the result                            
climateRecs.pprint()
#fireRecs.pprint()
#wordCounts.pprint()

ssc.start()
try:
    ssc.awaitTermination(timeout=60)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()

#ssc.stop()
#sc.stop()

-------------------------------------------
Time: 2018-10-07 16:58:55
-------------------------------------------
("'2018-10-07 16:58:53'", ["'cdata'", '948700', '19', '56.8', '7.9', '11.1', "' 0.00I'", "'2018-10-07 16:58:53'"])

-------------------------------------------
Time: 2018-10-07 16:59:00
-------------------------------------------
("'2018-10-07 16:58:56'", ["'cdata'", '948700', '15', '50.7', '9.2', '13.0', "' 0.02G'", "'2018-10-07 16:58:56'"])

-------------------------------------------
Time: 2018-10-07 16:59:05
-------------------------------------------
("'2018-10-07 16:58:59'", ["'cdata'", '948700', '16', '53.6', '8.1', '15.0', "' 0.00G'", "'2018-10-07 16:58:59'"])
("'2018-10-07 16:59:02'", ["'cdata'", '948700', '24', '61.6', '7.7', '14.0', "' 0.00I'", "'2018-10-07 16:59:02'"])

-------------------------------------------
Time: 2018-10-07 16:59:10
-------------------------------------------
("'2018-10-07 16:59:05'", ["'cdata'", '948700', '24', '62.3', '7.0', '13.0', "' 0

In [61]:
sc.stop()
ssc.stop()

-------------------------------------------
Time: 2018-10-07 17:00:25
-------------------------------------------
("'2018-10-07 17:00:20'", ["'cdata'", '948700', '20', '55.8', '10.5', '15.9', "' 0.01G'", "'2018-10-07 17:00:20'"])
("'2018-10-07 17:00:23'", ["'cdata'", '948700', '16', '48.4', '8.1', '15.9', "' 0.00G'", "'2018-10-07 17:00:23'"])

